Importazione delle librerie

In [ ]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go

Importazione dei dati

In [ ]:
dataset= pd.read_csv("./compitino_dataviz_2021_2022.csv")
dataset.head()

Calcolo colonne aggiuntive utili alla visualizzazione

In [ ]:
dataset["delta_score"] = dataset["score_postop"] - dataset["score_preop"] 
dataset["peggiorato"] = dataset["delta_score"] < 0
dataset["pshysical_score"] = dataset["nome_modulistica"] == "SF12 PhysicalScore"

Creazione di classi di età con numero di osservazioni omogeneo

In [ ]:
#creazione di 6 intervalli di età
dataset["gruppo_eta"] = pd.qcut(dataset["anni_ricovero"], 6)
#conversione della colonna appena creata in stringa, per comodità 
dataset["gruppo_eta"] = dataset["gruppo_eta"].astype(str)

Calcolo per ogni classe d'età del numero punteggi fisici migliorati e peggiorati

In [ ]:
#creazione nuovo dataframe
dataframe = pd.DataFrame({"gruppo_eta":[], "peggiorati": [], "migliorati":[]})
#ciclo for per ogni classe d'età
for item in dataset["gruppo_eta"].unique():
  #query per identificare i record con punteggio peggiorato per la classe d'età in esame
  queryStringPegg = "gruppo_eta == '"+str(item)+"' and peggiorato == 1 and pshysical_score == 1"
  #query per identificare i record con punteggio migliorato per la classe d'età in esame
  queryStringMigl = "gruppo_eta == '"+str(item)+"' and peggiorato == 0 and pshysical_score == 1"

  #query
  resPeggiorati = dataset.query(queryStringPegg)
  resMigliorati = dataset.query(queryStringMigl)
  #inserimento nel dataframe del numero di migliorati o peggiorati per la classe d'età in esame
  dataframe.loc[len(dataframe.index)] = [str(item), len(resPeggiorati), len(resMigliorati)]
#ordinamento delle classi d'età
dataframe = dataframe.sort_values("gruppo_eta")
#calcolo della percentuale di migliorati e peggiorati rispetto al totale di migliorati e peggiorati
dataframe["peggiorati"] =- (dataframe["peggiorati"]/ dataframe["peggiorati"].sum())
dataframe["migliorati"] = dataframe["migliorati"]/ dataframe["migliorati"].sum()
dataframe.head()


Creazione della visualizzazione a piramide relativa alla percentuale di migliorati e peggiorati per ogni classe di età

In [ ]:

#creazione finestra grafico
fig = go.Figure()
#creazione layout grafico con nome degli assi, titolo del grafico, etichette dei dati degli assi e grandezza del font
fig.update_layout(go.Layout(
                  yaxis=go.layout.YAxis(title='Classe di età'),
                  xaxis=go.layout.XAxis(
                    range=[-0.5,0.5],
                    tickvals=[-0.5, -0.25, 0 , 0.25, 0.5],
                    ticktext=[50,25, 0, 25, 50],
                    title='Percentuale (%)'),
                  barmode='overlay',
                  title="L'età non influisce sulla percentuale di score fisici peggiorati/migliorati",
                  bargap=0.1,
                  font=dict(
                     size=14,
                    )
                  )
                )
#creazione della porzione di piramide relativo alla percentuale di peggiorati
fig.add_trace(go.Bar(y= dataframe["gruppo_eta"].to_list(), x = dataframe["peggiorati"], 
               orientation='h',
               name='Peggiorati',
               text=(dataframe["peggiorati"]*100).abs().round(2).to_list(),
               hoverinfo='skip', 
               marker=dict(color='#F4B942'),
                textposition='auto',
               ),
              )
#creazione della porzione di piramide relativo alla percentuale di migliorati
fig.add_trace( go.Bar(y= dataframe["gruppo_eta"].to_list(), x = dataframe["migliorati"],
               orientation='h',
               text=(dataframe["migliorati"]*100).abs().round(2).to_list(),
               name='Migliorati',
                hoverinfo='skip',
               marker=dict(color='#4059AD'),
                textposition='auto',
               ))

#visualizzazione grafico
fig.show()

Salvataggio della visualizzazione come immagine

In [ ]:
#salvataggio della visualizzazione come file png con larghezza di 2000px
fig.write_image("prima_visualizzazione.png", scale=1, width= 2000 )